# Optimising the Rosenbrock Function

**Note**:  
This notebook is just getting started! Sorry for any messy code and lack of math rigor. If you'd like to help improve it, please feel free to dive in!

The Rosenbrock function is a classic non-convex optimization problem and a popular benchmark for testing optimization algorithms.

### Rosenbrock Function Formula 📝
The Rosenbrock function is defined as:

$$
f(x) = \sum_{i=1}^{n-1} [100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2]
$$

### Global Minimum 🎯
For $N=3$, the global minimum is at $x = [1, 1, 1]$. 

## Algorithms Exploration 🚀

So, here's my starting point: How do different optimization algorithms tackle this problem? Let's find out together!

Let's kick things off with these algorithms:
- **PyTorch**: Using its automatic differentiation and optimisation algorithms.
- **Firedrake.adjoint**: Leveraging its automatic differentiation combined with SciPy's optimization algorithms.

First up, we'll dive into `torch.optim`, a module packed with optimization algorithms.

In [ ]:
%time
from functools import partial
import torch
from torch import optim
# I got this example from https://discuss.pytorch.org/t/there-are-a-leverberg-marquardt-like-optimizer-for-pytorch/68055/3

def rosenbrock(x):
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

a = torch.tensor([0., 0., 0.]) + torch.rand(3)
a_init = [float(a[0]), float(a[1]), float(a[2])]
a.requires_grad_()
loss = partial(rosenbrock, a)


def print_iter(i, a, loss):
    print(f'{i} [{a[0]:.6f}, {a[1]:.6f}, {a[2]:.6f}] {loss:.6f}')


opt = optim.LBFGS([a], line_search_fn='strong_wolfe')
print_iter(0, a, loss())
torch_loss_interation = []
for i in range(24):
    opt.zero_grad()
    loss().backward()
    opt.step(loss)
    torch_loss_interation.append(loss())
    print_iter(i+1, a, loss())



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
0 [0.674467, 0.508772, 0.303139] 0.396131
1 [0.701306, 0.489729, 0.303139] 0.089660
2 [0.701351, 0.492088, 0.303139] 0.089195
3 [0.703105, 0.491983, 0.303139] 0.088710
4 [0.702848, 0.493639, 0.303139] 0.088312
5 [0.802620, 0.630874, 0.303139] 0.056713
6 [0.887788, 0.774882, 0.303139] 0.030239
7 [0.887788, 0.774883, 0.303139] 0.030237
8 [0.887788, 0.774883, 0.303139] 0.030236
9 [0.887788, 0.774884, 0.303139] 0.030235
10 [0.887788, 0.774884, 0.303139] 0.030234
11 [0.887788, 0.774885, 0.303139] 0.030232
12 [0.887788, 0.774885, 0.303139] 0.030231
13 [0.887788, 0.774886, 0.303139] 0.030230
14 [0.887788, 0.774886, 0.303139] 0.030229
15 [0.887788, 0.774887, 0.303139] 0.030227
16 [0.887788, 0.774887, 0.303139] 0.030226
17 [0.887788, 0.774888, 0.303139] 0.030225
18 [0.887788, 0.774888, 0.303139] 0.030223
19 [0.887788, 0.774889, 0.303139] 0.030222
20 [0.887788, 0.774889, 0.303139] 0.030221
21 [0.887788, 0.774890, 0.303139] 0.030220


You might (or might not) already know I'm a Firedrake developer—yep, that's me! So, naturally, I will be exploring the `firedrake.adjoint` module. This **powerful** tool combines Firedrake's automatic differentiation capabilities with SciPy's optimization algorithms, making it perfect for tackling PDE-constrained optimization problems. 🚀 

In [ ]:
%time
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["PETSC_ARCH"] = "minimal_petsc"
os.environ["PETSC_DIR"] = "/Users/ddolci/tes_fire_install/petsc"
from firedrake import *
from firedrake.adjoint import *
continue_annotation()

def test_optimisation_constant_control():
    """This tests a list of controls in a minimisation (through scipy L-BFGS-B)"""
    mesh = UnitSquareMesh(1, 1)
    R = FunctionSpace(mesh, "R", 0)

    n = 3
    x = [Function(R, val=float(a_init[i])) for i in range(n)]
    c = [Control(xi) for xi in x]
    
    f = sum(100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2 for i in range(n-1))

    J = assemble(f * dx(domain=mesh))
    rf = ReducedFunctional(J, c)
    # minimize is a method from pyadjoint.optimization
    result = minimize(rf, method="L-BFGS-B", options={"disp": True, "maxiter": 20})
    return [float(result[i]) for i in range(n)]
result = test_optimisation_constant_control()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.91 µs
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.33590D-01    |proj g|=  1.51835D+01

At iterate    1    f=  3.83680D-01    |proj g|=  6.05086D+00

At iterate    2    f=  3.16594D-01    |proj g|=  2.15273D+00

At iterate    3    f=  3.08543D-01    |proj g|=  5.83912D-01

At iterate    4    f=  3.02234D-01    |proj g|=  1.44090D+00

At iterate    5    f=  2.82629D-01    |proj g|=  2.53275D+00

At iterate    6    f=  1.65780D-01    |proj g|=  4.13653D+00

At iterate    7    f=  1.24142D-01    |proj g|=  5.51635D+00

At iterate    8    f=  9.22824D-02    |proj g|=  1.71032D+00

At iterate    9    f=  7.94258D-02    |proj g|=  3.51275D+00

At iterate   10    f=  5.82256D-02    |proj g|=  3.39924D+00

At iterate   11    f=  2.89178D-02    |proj g|=  3.32214D+00

At iterate   12    f=  1

 This problem is unconstrained.


e   14    f=  2.45635D-03    |proj g|=  1.07109D+00

At iterate   15    f=  1.17907D-03    |proj g|=  6.44250D-01

At iterate   16    f=  4.95791D-04    |proj g|=  5.42251D-01

At iterate   17    f=  6.54252D-05    |proj g|=  2.92141D-01

At iterate   18    f=  5.32572D-06    |proj g|=  4.25161D-02

At iterate   19    f=  6.93677D-07    |proj g|=  1.64864D-02

At iterate   20    f=  2.75169D-10    |proj g|=  3.44461D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     20     26      1     0     0   3.445D-04   2.752D-10
  F =   2.7516883153999110E-010

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 

In [ ]:
print("firedrake.adjoint result", result)
print("torch result", [float(a[0]), float(a[1]), float(a[2])])

firedrake.adjoint result [0.9999936706742869, 0.9999874921565328, 0.9999758584787991]
torch result [0.8877875208854675, 0.7748910784721375, 0.3031386137008667]


Great, we have a good start with two high-level algorithms! Let us compare their results. It seems like `firedrake.adjoint` combined with SciPy's optimization algorithms is more accurate than `torch` algorithms. But I'm not entirely sure if this comparison is fair... 🤔

 PyTorch. But I'm not entirely sure if this comparison is fair... 🤔

Next up, I will explore the `torch.optim`. Stay in loop for more updates! 🚀

Do you want to try to use Firedrake? Here is the [installation guide](https://www.firedrakeproject.org/download.html). 🌟

It requires some time to install, but it's worth it! You can also use a Docker image with Firedrake pre-installed. 😉 